In [7]:
import numpy as np

In [35]:
# Geometry
radius = 7       # m
A = np.pi * radius**2
print (A)

153.93804002589985


In [37]:
# Mass
mass = 10000000 #kg

In [39]:
# Depth limits (z = 0 at waterline, negative downward)
z_top = 0
z_bottom = -91

In [41]:
# Define zones (m)
zones = [
    {"z1": 0, "z2": -61, "rho": 300},    # Air-filled section
    {"z1": -61, "z2": -71, "rho": 1025}, # Water ballast
    {"z1": -71, "z2": -91, "rho": 5000}  # Solid ballast
]

In [43]:
Mz_total=0
M_total=0

for zone in zones:
    z1, z2, rho = zone["z1"], zone["z2"], zone["rho"]
    dz = abs(z2 - z1)

    M_zone = rho * A * dz     #mass of that zone(ballast,water,air)
    z_cg_zone = 0.5 * (z1 + z2)
    
    M_total += M_zone
    Mz_total += M_zone * z_cg_zone

z_total = Mz_total / M_total

print(f"Total mass platform={M_total} kg")
print(f"Total centroid platform={z_total} m")

Total mass platform=19788735.04532943 kg
Total centroid platform=-72.6149358226371 m


In [31]:
Height_tower=83 #m
Diameter_nacelle=5 #m
Height_narcelle=88

Blade_length= np.arange(0, 80, 10) #m

#Mass #kg
Mass_blade=300*Blade_length
Mass_nacelle=350000 
Mass_tower=670000

y_cg_list = []

for blade_length in Blade_length:
    
    y_cg_blade = 0.4*blade_length+Height_tower
    y_cg_tower = Height_tower/2
    y_cg_nacelle = Height_tower+(Diameter_nacelle/2)
    
    My_total=Mass_blade*y_cg_blade + Mass_nacelle*y_cg_nacelle + Mass_tower*Height_tower
    Mm_total=Mass_blade+Mass_nacelle+Mass_tower

    y_total = My_total/Mm_total

    y_cg_list.append(y_total)

print (f"total mass above water={Mm_total} kg")
print (f"y cg above water surface={y_total}m")





total mass above water=[1020000 1023000 1026000 1029000 1032000 1035000 1038000 1041000] kg
y cg above water surface=[83.85784314 83.93743891 84.0165692  84.0952381  84.17344961 84.25120773
 84.32851638 84.40537944]m


In [45]:
M_final=M_total+Mm_total
Mcg_final= M_total*(abs(z_bottom-z_total))+ (Mm_total*(y_total+abs(z_bottom)))

cg_final= Mcg_final/M_final

print (f"mass final={M_final} kg")
print (f"cg final from bottom platform={cg_final} m")

mass final=[20808735.04532943 20811735.04532943 20814735.04532943 20817735.04532943
 20820735.04532943 20823735.04532943 20826735.04532943 20829735.04532943] kg
cg final from bottom platform=[26.0550275  26.08038987 26.10574493 26.13109268 26.15643312 26.18176627
 26.20709212 26.23241067] m


In [47]:
# centre of buoyancy
# only account for the submerged part of the wind turbine (the spar buoy platform)

from scipy.integrate import quad

def volume_cylinder(h):
    return np.pi*(radius**2)*h

I, err = quad(volume_cylinder, z_top, z_bottom)

cb_final = I/volume_cylinder(z_top-z_bottom)

print(f"cb from bottom of platform = {cb_final} m")
    

cb from bottom of platform = 45.5 m


In [49]:
# finding longitudinal metacentric height (GM)

GM=cb_final-cg_final

print(f"GM={GM}m")

GM=[19.4449725  19.41961013 19.39425507 19.36890732 19.34356688 19.31823373
 19.29290788 19.26758933]m


In [70]:
# finding restoring moment of pitching movement

displaced_volume = np.pi*(radius**2)*(z_top-z_bottom)
seawater_density = 1025 #kg/m3
g = 9.81 #m/s2

Restoring_hydrostatic_moment = seawater_density*g*displaced_volume*GM

print (f"restoring hydrostatic moment (Kh)={Restoring_hydrostatic_moment}")

restoring hydrostatic moment (Kh)=[2.73897174e+09 2.73539926e+09 2.73182780e+09 2.72825738e+09
 2.72468799e+09 2.72111962e+09 2.71755228e+09 2.71398598e+09]


In [72]:
#Setting parameters for restoring moment from mooring line 

n=3
k=5.05*(10^5)
horizontal_displacement=20 #m|
vertical_displacement=70 #m

T_i=k*horizontal_displacement
for T in range (n):
    Restoring_moment_mooring += (T_i)*vertical_displacement

    
print(f"Restoring_moment_mooring={Restoring_moment_mooring}" )

Restoring_moment_mooring=2121318150.0


In [74]:
"""
Moment of Inertia Calculator for Spar Offshore Wind Turbine
-----------------------------------------------------------

I_total = I_platform + I_rotor + I_added

This script estimates:
- Platform moment of inertia
- Rotor moment of inertia
- Added (hydrodynamic) inertia
and sums them for the total system inertia.

All units: SI (kg, m, kg·m²)
"""

import math

# 1. Platform Moment of Inertia
def calculate_platform_inertia(mass_platform, radius_platform, height_platform=None):
    """
    Calculate the platform's rotational inertia about its central (pitch/roll) axis.
    By default, assumes a solid cylinder rotating about its central axis.

    I_platform = 0.5 * m * r^2

    If height_platform is provided, you can use it to adjust the axis (optional).
    """
    I_platform = 0.5 * mass_platform * radius_platform**2
    return I_platform



# 2. Rotor Moment of Inertia
def calculate_rotor_inertia(rotor_mass, rotor_radius):
    """
    Calculate the rotor's moment of inertia assuming a solid disk (blades + hub).

    I_rotor = 0.5 * m * r^2
    """
    I_rotor = 0.5 * rotor_mass * rotor_radius**2
    return I_rotor


# 3. Added (Hydrodynamic) Inertia
def calculate_added_inertia(added_mass, reference_radius):
    """
    Calculate the added (hydrodynamic) moment of inertia.

    I_added = added_mass * r^2
    """
    I_added = added_mass * reference_radius**2
    return I_added


# 4. Total Moment of Inertia
def total_inertia(I_platform, I_rotor, I_added):
    """
    Sum the components.
    """
    return I_platform + I_rotor + I_added


# Example Usage
if __name__ == "__main__":
    # Example parameters (replace with real design data)
    mass_platform = 8.0e6       # kg
    radius_platform = 6.5       # m
    mass_rotor = 1.1e5          # kg
    radius_rotor = 63.0         # m
    added_mass = 1.5e6          # kg (hydrodynamic added mass)
    added_radius = 6.5          # m (approx same as platform radius)

    # Calculate each component
    I_platform = calculate_platform_inertia(mass_platform, radius_platform)
    I_rotor = calculate_rotor_inertia(mass_rotor, radius_rotor)
    I_added = calculate_added_inertia(added_mass, added_radius)

    # Total
    I_total = total_inertia(I_platform, I_rotor, I_added)

    # Print results
    print("\n=== Offshore Spar Wind Turbine Inertia Results ===")
    print(f"Platform Inertia: {I_platform:,.2e} kg·m²")
    print(f"Rotor Inertia:    {I_rotor:,.2e} kg·m²")
    print(f"Added Inertia:    {I_added:,.2e} kg·m²")
    print("----------------------------------------------")
    print(f"Total Inertia:    {I_total:,.2e} kg·m²\n")



=== Offshore Spar Wind Turbine Inertia Results ===
Platform Inertia: 1.69e+08 kg·m²
Rotor Inertia:    2.18e+08 kg·m²
Added Inertia:    6.34e+07 kg·m²
----------------------------------------------
Total Inertia:    4.51e+08 kg·m²



In [112]:
#Finding hydrodynamic damping

import math 
import pandas as pd

Restoring_hydrostatic_moment=np.array([2.73897174e+09,2.73539926e+09,2.73182780e+09,2.72825738e+09,2.72468799e+09,2.72111962e+09,2.71755228e+09,2.71398598e+09])
hydrodynamic_damping_list=[]

delta=np.arange(0,1,0.2)

for d in np.arange(0,1,0.2):
    Ch_per_Kh=[] #damping list for every restoring hydrostatic moment
    for Kh in Restoring_hydrostatic_moment:
        hydrodynamic_damping=2*d*math.sqrt(Kh* I_total) 
        Ch_per_Kh.append(hydrodynamic_damping) 
    hydrodynamic_damping_list.append(Ch_per_Kh)

hydrodynamic_damping_array=np.array(hydrodynamic_damping_list)

# Create column names dynamically
columns = [f"Ch{i+1}" for i in range(len(Restoring_hydrostatic_moment))]

# Combine damping ratio + Ch values into DataFrame
df = pd.DataFrame(hydrodynamic_damping_array, columns=columns)
df.insert(0, "Damping Ratio", delta)

# Print nicely
print(df.to_markdown(index=False, floatfmt=".2e"))



|   Damping Ratio |      Ch1 |      Ch2 |      Ch3 |      Ch4 |      Ch5 |      Ch6 |      Ch7 |      Ch8 |
|----------------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|        0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 | 0.00e+00 |
|        2.00e-01 | 4.44e+08 | 4.44e+08 | 4.44e+08 | 4.44e+08 | 4.43e+08 | 4.43e+08 | 4.43e+08 | 4.42e+08 |
|        4.00e-01 | 8.89e+08 | 8.88e+08 | 8.88e+08 | 8.87e+08 | 8.86e+08 | 8.86e+08 | 8.85e+08 | 8.85e+08 |
|        6.00e-01 | 1.33e+09 | 1.33e+09 | 1.33e+09 | 1.33e+09 | 1.33e+09 | 1.33e+09 | 1.33e+09 | 1.33e+09 |
|        8.00e-01 | 1.78e+09 | 1.78e+09 | 1.78e+09 | 1.77e+09 | 1.77e+09 | 1.77e+09 | 1.77e+09 | 1.77e+09 |
